In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline
%config Completer.use_jedi = False

In this notebook, I'm going to implement the formulas for predicting the ratings of movies using the Item-based neighborhood models.

The provided implementation is optimized for understandability, rather than being optimized for comuptational efficiency.

Let's start by recreating the ratings dataset as provided in book in table 2.1. To make our lives easier, we'll use Pandas dataframe.

In [4]:
data = [
    [1, 7, 6, 7, 4, 5, 4],
    [2, 6, 7, np.NaN, 4, 3, 4],
    [3, np.NaN, 3, 3, 1, 1, np.NaN],
    [4, 1, 2, 2, 3, 3, 4],
    [5, 1, np.NaN, 1, 2, 3, 3]
]


ratings = pd.DataFrame(data, columns=['userId', 'm_1', 'm_2', 'm_3', 'm_4', 'm_5', 'm_6'])
ratings = ratings.set_index('userId')
ratings

,m_1,m_2,m_3,m_4,m_5,m_6
userId,,,,,,
1,7.0,6.0,7.0,4,5,4.0
2,6.0,7.0,NaN,4,3,4.0
3,NaN,3.0,3.0,1,1,NaN
4,1.0,2.0,2.0,3,3,4.0
5,1.0,NaN,1.0,2,3,3.0


There. It's exactly the same as it is in the book. Let's normalize tha ratings now. Normalization is done by mean-centering the rows (i.e. for each cell, subtract the mean value of that row). What this is effectively doing is mean-centering users ratings.

In [5]:
normalized_ratings = ratings.apply(lambda x: x - x.mean(), axis=1)
normalized_ratings

,m_1,m_2,m_3,m_4,m_5,m_6
userId,,,,,,
1,1.5,0.5,1.5,-1.5,-0.5,-1.5
2,1.2,2.2,NaN,-0.8,-1.8,-0.8
3,NaN,1.0,1.0,-1.0,-1.0,NaN
4,-1.5,-0.5,-0.5,0.5,0.5,1.5
5,-1.0,NaN,-1.0,0.0,1.0,1.0


Nice and sweet. So all rows are mean-centered now.

In order to efficiently compute the cosine similarities, we will replace the NaN values with zeroes. For anyone wondering - no, this does not affect the similarities in any way.

In [6]:
normalized_ratings = normalized_ratings.fillna(0)
normalized_ratings

,m_1,m_2,m_3,m_4,m_5,m_6
userId,,,,,,
1,1.5,0.5,1.5,-1.5,-0.5,-1.5
2,1.2,2.2,0.0,-0.8,-1.8,-0.8
3,0.0,1.0,1.0,-1.0,-1.0,0.0
4,-1.5,-0.5,-0.5,0.5,0.5,1.5
5,-1.0,0.0,-1.0,0.0,1.0,1.0


Much better. Now the 0.0 ratings are set for missing ratings, and everything else is mean centered. Next step is calculation of cosine similarity coefficients, also referred to as "adjusted cosine similarities". The adverb "adjusted" is used because we are calculating coefficients against a previously mean-centered matrix.

We will not code the formula 2.14 by hand and will rather rely on using the SciKit's `cosine_similarity()` function. This function calculates the the cosine similarities between the rows in the matrix. Since we need coefficients for items, we will first transpose the matrix and then calculate the coefficients. 

In [8]:
# Transposing the matrix switches columns and rows
normalized_ratings.T

userId,1,2,3,4,5
m_1,1.5,1.2,0.0,-1.5,-1.0
m_2,0.5,2.2,1.0,-0.5,0.0
m_3,1.5,0.0,1.0,-0.5,-1.0
m_4,-1.5,-0.8,-1.0,0.5,0.0
m_5,-0.5,-1.8,-1.0,0.5,1.0
m_6,-1.5,-0.8,0.0,1.5,1.0


In [9]:
# Calculate similarity coefficients now
similarity_coefficients = cosine_similarity(normalized_ratings.T)
similarity_coefficients[:, :5]

array([[ 1.        ,  0.62413132,  0.71577084, -0.73878026, -0.73832952],
       [ 0.62413132,  1.        ,  0.3744373 , -0.73391041, -0.90509063],
       [ 0.71577084,  0.3744373 ,  1.        , -0.81088939, -0.59028134],
       [-0.73878026, -0.73391041, -0.81088939,  1.        ,  0.70567109],
       [-0.73832952, -0.90509063, -0.59028134,  0.70567109,  1.        ],
       [-0.9896203 , -0.522503  , -0.76097353,  0.72196647,  0.66367597]])

That was easy. Now, for sake of faster lookups and general ease of use, we'll convert this matrix to Pandas dataframe. You'll see later one why I found this as more useful approach.

In [10]:
# Convert similarity coefficients to DataFrame for easier searching
similarity_coefficients = pd.DataFrame(index=normalized_ratings.columns, columns=normalized_ratings.columns, data=similarity_coefficients)
similarity_coefficients

,m_1,m_2,m_3,m_4,m_5,m_6
m_1,1.000000,0.624131,0.715771,-0.738780,-0.738330,-0.989620
m_2,0.624131,1.000000,0.374437,-0.733910,-0.905091,-0.522503
m_3,0.715771,0.374437,1.000000,-0.810889,-0.590281,-0.760974
m_4,-0.738780,-0.733910,-0.810889,1.000000,0.705671,0.721966
m_5,-0.738330,-0.905091,-0.590281,0.705671,1.000000,0.663676
m_6,-0.989620,-0.522503,-0.760974,0.721966,0.663676,1.000000


There. We made it so that both row and columns indices are items. This will allow us to quickly search for similarity between X and Y items, e.g. `similarity_coefficients.loc['m_1']['m_3']` will provide us with adjusted similarity coefficient between movies 1 and 3.

Next step is defining the function that will return the movies similar to target movie. This is actually pretty easy. All we need to do is to take the similarity coefficients for given movie, sort them in descending order (so that highest coefficients come first) and then return the top-k results.

In [11]:
def get_similar_movies(target_movie, k=10):
    '''Returns list of top-k movies similar to target movie'''
    
    # Get all coefficients for the target movie
    similar_movies = similarity_coefficients.loc[target_movie]
    
    # Drop the target_movies as we don't want to report the Target Movie as being similar to Target Movie
    similar_movies = similar_movies.drop(target_movie)
    
    # Sort in Descending order. More similar ones should come first
    similar_movies = similar_movies.sort_values(ascending=False)
    
    # Leave only the ones that are positively correlated with target movie
    similar_movies = similar_movies[similar_movies > 0]
    
    # Return top-k results
    return similar_movies.head(k)

Cool. Let's see how that works

In [13]:
get_similar_movies('m_1')

m_3    0.715771
m_2    0.624131
Name: m_1, dtype: float64

In [14]:
get_similar_movies('m_5')

m_4    0.705671
m_6    0.663676
Name: m_5, dtype: float64

So what this telling us is that, for move 1, movies 2 and 3 seem to be similar. For movie 5, movies 4 and 6 are most similar. This matches exactly to what is written in the book.

Let's write a function for predicting the ratings now. We will be using formula 2.15 for this

In [15]:
def predict_rating(target_user, target_movie):
    '''Predicts rating of target user for target movie'''

    # First, find the movies that are similar to target movie
    similar_movies = get_similar_movies(target_movie, k = 100)
    
    # Out of those that are similar, leave only the ones that were rated by given user
    similar_movies_that_were_rated_by_user = ratings.loc[target_user][similar_movies.index].dropna()
    
    # Similarity coefficients between the target movie and the movies that are similar to it
    cosine_similarities = similarity_coefficients.loc[target_movie][similar_movies_that_were_rated_by_user.index]
    
    # Target users ratings of movies that are similar to target movie
    ratings_for_similar_movies = ratings.loc[target_user][similar_movies_that_were_rated_by_user.index]
    
    # Calculate the numerator part of formula 2.15
    # Basically, numerator is calculated by multiplying similarities coefficients and users rating for similar movies
    # You can think of it as saying - multiply how similar the target movie is and how user rated the movie similar
    # to it, and then sum up all those results
    numerator = sum(cosine_similarities * ratings_for_similar_movies)
    
    # Denominator is just the sum of similarity coefficients
    denominator = sum(cosine_similarities)

    return numerator / denominator

As can be seen, the rating of specific movie for specific user is predicted by finding the similar movies first, then filtering out and leaving only ones that were already rated by target user. Finally, the rating is predicted by multiplying similarity coefficients and predicted ratings and dividing that by sum of similarity coefficients.

Again, this matches the formula 2.15 from the book. I honestly have no clue if there is some universal name for this formula, so I'll refer to it as "rating prediction formula".

What the book claims is that the rating for user 3 for movie 1 should be `3` and for movie 6 should be `1`. Let's try that out:

In [17]:
predict_rating(3, 'm_1')

3.0000000000000004

In [18]:
predict_rating(3, 'm_6')

1.0

Perfect! Seems to match exactly to what the book says :)

So, how do we implement the full recommender system now? Well, it's actually rather easy. For each user, we calculate the top-k movies that have highest predicted rating and we report those to user as "suggested" movies. One cool benefit of the Item-based approach is that we can also explain WHY is something predicted (e.g. you might like movie X because you liked movie Y).

From my understanding, these calculations of similar movies are usually precalculated upfront (i.e. in the background). I would assume that service providers (e.g. Netflix) usually precalculate and cache the top-100 recommended movies and then use those cached results for recommendations. But, again, that's just my layman assumption.

Anyway, let's implement the actual recommender system.

In [29]:
ratings.loc[3][ratings.loc[3].isna()].index

Index(['m_1', 'm_6'], dtype='object')

In [65]:
import operator

def recommend_movies(target_user, k = 3):
    '''Recommends top-k movies to target user'''
    
    # First - let's see which user didn't rate yet
    is_unrated = ratings.loc[target_user].isna()
    
    unrated_movies = ratings.loc[target_user][is_unrated].index
    
    if len(unrated_movies) == 0:
        # Well, seems like there's nothing we can recommend as user watched all movies we have
        return []
    
    predicted_ratings = {}
    
    # Predict rating for each unrated movie
    for movie in unrated_movies:
        predicted_ratings[movie] = predict_rating(target_user, movie)
        
    # Sort the dictionary by values and return keys (movie IDs)
    sorted_predicted_ratings = sorted(predicted_ratings.items(), key=operator.itemgetter(1), reverse=True)
    
    positively_predicted = []
    
    for row in sorted_predicted_ratings:
        movie_id = row[0]
        predicted_rating = row[1]
        
        if predicted_rating >= 3:
            positively_predicted.append(movie_id)
    
    return positively_predicted[:k]

Finally, let's see which movies are predicted for user 3. Based on the previously seen data, the only movie that has positive predicted rating is movie 1.

In [66]:
recommend_movies(3)

['m_1']

And that's exactly what we've got. Movie 1 seems to be only movie predicted for user 3.

In [67]:
recommend_movies(1)

[]

No movies predicted for user 1 because he seems to have watched all movies.

Finally, let's see what happens when we introduce a new movie to the matrix. This movie wasn't rated by anyone so far.

In [71]:
ratings['m_7'] = np.nan
ratings

,m_1,m_2,m_3,m_4,m_5,m_6,m_7
userId,,,,,,,
1,7.0,6.0,7.0,4,5,4.0,NaN
2,6.0,7.0,NaN,4,3,4.0,NaN
3,NaN,3.0,3.0,1,1,NaN,NaN
4,1.0,2.0,2.0,3,3,4.0,NaN
5,1.0,NaN,1.0,2,3,3.0,NaN


We need to append it to normalized_ratings matrix as well. Since all values are NaN, we can just insert it with value of 0.

In [77]:
normalized_ratings['m_7'] = 0
normalized_ratings

,m_1,m_2,m_3,m_4,m_5,m_6,m_7
userId,,,,,,,
1,1.5,0.5,1.5,-1.5,-0.5,-1.5,0
2,1.2,2.2,0.0,-0.8,-1.8,-0.8,0
3,0.0,1.0,1.0,-1.0,-1.0,0.0,0
4,-1.5,-0.5,-0.5,0.5,0.5,1.5,0
5,-1.0,0.0,-1.0,0.0,1.0,1.0,0


Let's recalculate the similarity coefficients now.

In [80]:
similarity_coefficients = cosine_similarity(normalized_ratings.T)

similarity_coefficients = pd.DataFrame(index=normalized_ratings.columns, columns=normalized_ratings.columns, data=similarity_coefficients)

similarity_coefficients

,m_1,m_2,m_3,m_4,m_5,m_6,m_7
m_1,1.000000,0.624131,0.715771,-0.738780,-0.738330,-0.989620,0.0
m_2,0.624131,1.000000,0.374437,-0.733910,-0.905091,-0.522503,0.0
m_3,0.715771,0.374437,1.000000,-0.810889,-0.590281,-0.760974,0.0
m_4,-0.738780,-0.733910,-0.810889,1.000000,0.705671,0.721966,0.0
m_5,-0.738330,-0.905091,-0.590281,0.705671,1.000000,0.663676,0.0
m_6,-0.989620,-0.522503,-0.760974,0.721966,0.663676,1.000000,0.0
m_7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [81]:
get_similar_movies('m_7')

Series([], Name: m_7, dtype: float64)

Well, that's exactly the problem that Collaborative filtering suffers from - cold start. Since this movie does not have ratings (because, it's new one) it will never be predicted to anyone.

Solving this is a topic on it's own, but I'll leave you here to theoretize and eventually come up with your own solution.

Cheers!